In [10]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
# from langchain_community.llms import OpenAI
# from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

In [42]:
!pip install -U openai

In [43]:
from langchain.llms import OpenAI

In [2]:
import os
os.environ['OPENAI_API_KEY'] = '...'

In [5]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}

In [20]:
# define a custom args_schema to provide more information about inputs.
class ShellInput(BaseModel):
    cmd: str = Field(description="Shell command")

def execute(cmd: str) -> str:
    """Executing the shell command"""
    result = '''Sat Jan  6 03:35:19 PM +04 2024
(base) alex@pc:~$'''
    return '\n'+result

shell_exec = StructuredTool.from_function(
    func=execute,
    name="Shell executor",
    description="Executing the provided shell command",
    args_schema=ShellInput,
    return_direct=True,
    # coroutine= ... <- you can specify an async method if desired as well
)

# Test
shell_res = shell_exec.run({"cmd":"date"})
print(shell_res)

# Define tools
tools = [shell_exec]


Sat Jan  6 03:35:19 PM +04 2024
(base) alex@pc:~$


In [52]:
# Choose the LLM to use
# llm = OpenAI()
# llm = OpenAI(model_version='gpt-4-1106-preview')
# llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

# https://platform.openai.com/docs/models/model-endpoint-compatibility
llm = OpenAI(model="gpt-3.5-turbo-instruct") # /v1/completions (Legacy)


# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

In [53]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [54]:
agent_executor.invoke({"input": "Check the current date please and answer what is a day today."})



> Entering new AgentExecutor chain...
 We need to know the current date to determine what day it is.
Action: Shell executor
Action Input: "date"
Sat Jan  6 03:35:19 PM +04 2024
(base) alex@pc:~$


> Finished chain.


{'input': 'Check the current date please and answer what is a day today.',
 'output': '\nSat Jan  6 03:35:19 PM +04 2024\n(base) alex@pc:~$'}